In [1]:
from bs4 import BeautifulSoup
import requests
import urllib
import re
import os

In [29]:
#find unique extensions before iterations 
def unique_ext(path):
    """
    It returns the unique extensions of different file recursively for given path.

    Parameters:
    ----------
    path: string
        folder path for directory

    Returns:
    --------
    unique: list 
        list of extensions

    """
    unique = set()
    for root, subFolder, files in os.walk(path):
        for item in files:
            ext = os.path.splitext(item)[1] 
            unique.add(ext)
    return list(unique)


def find_files(extension_list,path):
    """
    It returns the file names of given extensions.

    Parameters:
    ----------
    path: extension_list
        list of extensions

    Returns:
    --------
    files: list 
        list of file names

    """
    files_list=[]
    for root, subFolder, files in os.walk(path):
 
        for item in files:
            ext = os.path.splitext(item)[1]
            if(ext in extension_list): 
                file_list.append((item,root))
    return file_list

def create_relative_paths(files_list,base_path):
    rel_path = []
    for files_name,path in file_list:
        relative_path = path.replace(base_path+'/','') + files_name
        rel_path.append(relative_path)
    return rel_path
def get_real_name(raw_name):
    raw_name = "".join(raw_name.split('.')[:-1])
    pat = "(.+)(\d\d\d\d)(.+)"
    try:
        raw_name = re.sub('[^A-Za-z0-9]+', ' ', raw_name)
        print(raw_name)
        f = re.findall(pat,raw_name)
        name = "{0} ({1})".format(f[0][0] ,f[0][1])
    except:
        name = raw_name[:30]
    return name


In [30]:
def get_imdb_url(name):
    """
    It returns imdb url of movie.
    Parameters:
    ----------
    name: string 
        name of movie
    Returns:
    --------
    imdb_url:string
        imdb url
    """
    name = name +  " imdb"
    name = urllib.parse.quote(name)
    url = "https://google.com/search?q=" + name
    try:
        resp =requests.get(url)
        soup = BeautifulSoup(resp.text)
        string = str(resp.text)
        pat = "https://www.imdb.com/title/tt(\d+)"
        imdb_url = "https://www.imdb.com/title/tt" + re.findall(pat,string)[0]
    except Exception as e :
        print(e)
        imdb_url =None
    return imdb_url

In [31]:
def get_imdb_rating(url):
    """
    It returns the rating, summery and Crew.
    
    Parameters:
    ----------
    url: string
        url of imdb
        
    Returns:
    --------
    data :dict
        dictionary format
        data = {
            "rating":string,
            "crew":string,
            "summery":string
        }
    """

    data = {}
    
    try:
        resp =requests.get(url)
        soup = BeautifulSoup(resp.text)
    except:
        pass
    try:
        rating = re.findall('title="(.+?)"',str(soup.find_all(class_='ratingValue')[0]))[0]
        data["rating"] = rating
    except:
        data["rating"] = "NO IMDB RATING"
    
    try:
        summery = soup.find(class_='plot_summary')

        #removing spcial character from summery text
        summery_text = summery.find(class_="summary_text").text
        summery_text = re.sub('[^A-Za-z0-9:]+', ' ', summery_text)

        crew = []
        for x in summery.find_all(class_="credit_summary_item"):

            #removing spcial character from crew items
            x = re.sub('[^A-Za-z0-9:]+', ' ', x.text)
            crew.append(x)
        data["summery"] = summery_text
        data["crew"] = crew
    except:
        data["summery"] = "NO"
        data["crew"] = "NO"
        
    return data

In [32]:
# get_imdb_rating(url)

In [33]:
base_path = "/media/ayrus/Media/Movies"
#finding  with paths
print(unique_ext(base_path))
extension_list = ['.MKV', '.mkv', '.avi', '.MP4', '.mp4','.webm', '.WEBM']
file_list = []
paths = []
PATH = "/media/ayrus/Media/Movies/Hollywood/"
file_list += find_files(extension_list,PATH)

PATH = "/media/ayrus/Media/Movies/Hollywood Movies BRip-HD/"
file_list += find_files(extension_list,PATH)

PATH = "/media/ayrus/Media/Movies/Hollywood HD  Movies [English]/"
file_list += find_files(extension_list,PATH)


PATH = "/media/ayrus/Media/Movies/Hollywood Movies BRip-HD/"
file_list += find_files(extension_list,PATH)

PATH = "/media/ayrus/Media/Movies/Bollywood/"
file_list += find_files(extension_list,PATH)

# file_list = list(set(file_list))
file_list = sorted(file_list)
relative_paths = create_relative_paths(file_list,base_path)

total_m = len(set(file_list))

['.zip', '.ini', '.webm', '.avi', '.MKV', '.png', '.MP4', '.html', '.txt', '.pdf', '.jpg', '.nfo', '.srt', '.WEBM', '.py', '.dthumb', '.mp4', '.mkv']


In [37]:
layout = """
    <a href="{rel_path}">{name}</a><br>
    <b>IMDB rating:<a href="{url}">{rating}</a></b>
    <p><b>Summery:</b>{summery} </p>
    <p><b>Crew:</b><br> {crew}</p><br><br>
"""
struct = {}
a = set()
num = 1
with open("out.html","w") as f:
    for i, name in enumerate(file_list):

            name = get_real_name(name[0])
            try:
                struct[name]
            except:
                print(num,total_m,name)
                num +=1
                rel_path = relative_paths[i]
                url = get_imdb_url(name)
                data = get_imdb_rating(url)
                meta_data = layout.format(rel_path=rel_path,
                                          url = url,
                                          name = name,
                                          rating = data["rating"],
                                          summery=data["summery"],
                                          crew = "<br>".join(data["crew"]))
                struct[name] = [url,data]  

                f.write(meta_data)
       

1 X Men
1 278 1 X Men


/home/ayrus/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/ayrus/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
1 X Men
2 X Men United
2 278 2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
2 X Men United
3 X Men The Last Stand
3 278 3 X Men The Last Stand
3 X Men The Last Stand
3 X Men The Last Stand
3 X Men The Last Stand
3 X Men The Last Stand
3 X Men The Last Stand
3 X Men The Last Stand
3 X Men The Last Stand
3 X Men The Last Stand
3 X Men The Las

After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
After Earth 2013 720p Dual Audio Hindi Eng
Airlift
12 278 Airlift
Airlift
Alice in Wonderland
13 278 Alice in Wonderland
Alice in Wonderland
Alice in Wonderland
Alice in Wonderland
Alice in Wonderland
Alice in Wonderland
Alice

Batman vs Superman Dawn Of Justice 2016 720p HDTS x264 English Hindi Exclusive
Batman vs Superman Dawn Of Justice 2016 720p HDTS x264 English Hindi Exclusive
Batman vs Superman Dawn Of Justice 2016 720p HDTS x264 English Hindi Exclusive
Batman vs Superman Dawn Of Justice 2016 720p HDTS x264 English Hindi Exclusive
Batman vs Superman Dawn Of Justice 2016 720p HDTS x264 English Hindi Exclusive
Batman vs Superman Dawn Of Justice 2016 720p HDTS x264 English Hindi Exclusive
Batman vs Superman Dawn Of Justice 2016 720p HDTS x264 English Hindi Exclusive
Batman vs Superman Dawn Of Justice 2016 720p HDTS x264 English Hindi Exclusive
Batman vs Superman Dawn Of Justice 2016 720p HDTS x264 English Hindi Exclusive
Batman vs Superman Dawn Of Justice 2016 720p HDTS x264 English Hindi Exclusive
Batman vs Superman Dawn Of Justice 2016 720p HDTS x264 English Hindi Exclusive
Batman vs Superman Dawn Of Justice 2016 720p HDTS x264 English Hindi Exclusive
Batman vs Superman Dawn Of Justice 2016 720p HDTS x2

...

The5thWave2016720pBluRayx264VPPV
The5thWave2016720pBluRayx264VPPV
The5thWave2016720pBluRayx264VPPV
The5thWave2016720pBluRayx264VPPV
The5thWave2016720pBluRayx264VPPV
The5thWave2016720pBluRayx264VPPV
The5thWave2016720pBluRayx264VPPV
TheFinestHours2016720pBluRayx264 SPARKS
210 278 TheFinestHours201 (6720)
TheFinestHours2016720pBluRayx264 SPARKS
TheFinestHours2016720pBluRayx264 SPARKS
TheFinestHours2016720pBluRayx264 SPARKS
TheFinestHours2016720pBluRayx264 SPARKS
TheFinestHours2016720pBluRayx264 SPARKS
TheFinestHours2016720pBluRayx264 SPARKS
TheFinestHours2016720pBluRayx264 SPARKS
TheImitationGame2014HDRip
211 278 TheImitationGame (2014)
TheImitationGame2014HDRip
TheImitationGame2014HDRip
TheImitationGame2014HDRip
TheImitationGame2014HDRip
TheImitationGame2014HDRip
TheImitationGame2014HDRip
TheImitationGame2014HDRip
TheJungleBook20161080pBluRay6CHShAaNiGDooblekingmovie
212 278 TheJungleBook2016 (1080)
TheJungleBook20161080pBluRay6CHShAaNiGDooblekingmovie
TheJungleBook20161080pBluRay6CHShAa

Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Unstoppable 2010 Dual Audio Eng Hindi 
Uvaa 2015 DvDScr by Filmywap
223 278 Uvaa  (2015)
Uvaa 2015 DvDScr by Filmywap
Van Helsing 2004720pDual Audio English Hindi BRRipaaaevilacharya
224 278 Van Helsing 200 (4720)
Van Helsing 2004720pDual Audio English Hindi BRRipaaaevilacharya
Van Helsing 2004720

X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 2 BRip 720p Hindi Eng
X MEN 3
236 278 X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X MEN 3
X Men Apocalypse 2016 720p BluRay DD 51 Hindi English x264 Phantoms 
237 278 X Men Apocalypse  (2016)
X Men Apocalypse 2016 720p BluRay DD 51 Hindi English x264 Phantoms 
X Men Apocalypse 2016 720p BluRay DD 51 Hindi English x264 Phantoms 
X Men Apocalypse 2016 720p BluRay DD 51 Hindi Englis

quantico102hdtv lol ettv 
quantico102hdtv lol ettv 
quantico102hdtv lol ettv 
quantico102hdtv lol ettv 
quantico102hdtv lol ettv 
quantico102hdtv lol ettv 
quantico102hdtv lol ettv 
quantico104hdtv lol ettv 
245 278 quantico104hdtv lol ettv 
quantico104hdtv lol ettv 
quantico104hdtv lol ettv 
quantico104hdtv lol ettv 
quantico104hdtv lol ettv 
quantico104hdtv lol ettv 
quantico104hdtv lol ettv 
quantico104hdtv lol ettv 
quantico105hdtv lol ettv 
246 278 quantico105hdtv lol ettv 
quantico105hdtv lol ettv 
quantico105hdtv lol ettv 
quantico105hdtv lol ettv 
quantico105hdtv lol ettv 
quantico105hdtv lol ettv 
quantico105hdtv lol ettv 
quantico105hdtv lol ettv 
quantico106hdtv lol ettv 
247 278 quantico106hdtv lol ettv 
quantico106hdtv lol ettv 
quantico106hdtv lol ettv 
quantico106hdtv lol ettv 
quantico106hdtv lol ettv 
quantico106hdtv lol ettv 
quantico106hdtv lol ettv 
quantico106hdtv lol ettv 
quantico107hdtv lol ettv 
248 278 quantico107hdtv lol ettv 
quantico107hdtv lol ettv 
quanti

KeyboardInterrupt: 

In [38]:
import pickle
with open("data1.pkl","wb") as f:
    pickle.dump(struct,f)

In [13]:
total_m

278

In [14]:
# struct

{'1': ['https://www.imdb.com/title/tt2375559',
  {'crew': [' Director: Sukumar ',
    ' Writers: Jakka Hariprasad story Hari Prasad Jakka story 6 more credits ',
    ' Stars: Mahesh Babu Kriti Sanon Nassar See full cast crew '],
   'rating': '8.4 based on 39,671 user ratings',
   'summery': ' A rock star must overcome his psychological inhibitions to seek revenge for his parents death '}],
 '2': ['https://www.imdb.com/title/tt5813916',
  {'crew': [' Director: Alper Caglar ',
    ' Writer: Alper Caglar ',
    ' Stars: Ozan Aga Bedii Akin Murat Arkin See full cast crew '],
   'rating': '9.5 based on 96,667 user ratings',
   'summery': ' In a desolate war zone where screams of the innocent echo on the very line between disaster and valor 7 Maroon Berets will dance with death '}],
 '3': ['https://www.imdb.com/title/tt4922804',
  {'crew': [' Creator: Pedro Aguilera ',
    ' Stars: Jo o Miguel Bianca Comparato Vaneza Oliveira See full cast crew '],
   'rating': '7.5 based on 14,178 user rati

In [17]:
relative_paths

['Bollywood/Airlift.mkv',
 'Bollywood/Airlift.mkv',
 'Bollywood/Baaghi- 2016.mkv',
 'Bollywood/Baaghi- 2016.mkv',
 'Bollywood/Baahubali 2- The Conclusion DvDRip HD.mp4.mp4',
 'Bollywood/Baahubali 2- The Conclusion DvDRip HD.mp4.mp4',
 'Bollywood/Baahubali.The.Beginning.2015.720p.2CH.x265.HEVC-[My-Film].mkv',
 'Bollywood/Baahubali.The.Beginning.2015.720p.2CH.x265.HEVC-[My-Film].mkv',
 'Bollywood/Bajiro Mastani.mkv',
 'Bollywood/Bajiro Mastani.mkv',
 'Bollywood/Brothers.2015.720p.2CH.x265.HEVC-[My-Film].mkv',
 'Bollywood/Brothers.2015.720p.2CH.x265.HEVC-[My-Film].mkv',
 'Bollywood/Bullet Raja (2013) New Source 1CD Dvdscr X264 MP3 [D3Si MaNiACs]Bullet Raja (2013) New Source 1CD Dvdscr X264 MP3 [D3Si MaNiACs].mkv',
 'Bollywood/Bullet Raja (2013) New Source 1CD Dvdscr X264 MP3 [D3Si MaNiACs]Bullet Raja (2013) New Source 1CD Dvdscr X264 MP3 [D3Si MaNiACs].mkv',
 'Bollywood/Bullet Raja (2013) New Source 1CD Dvdscr X264 MP3 [D3Si MaNiACs]Sample.mkv',
 'Bollywood/Bullet Raja (2013) New Source 1